In [ ]:
from google.colab import files
uploaded = files.upload()

Saving conull.csv to conull.csv


In [ ]:
!pip install transformers datasets seqeval



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=ec455307fac09c30034d6afadc9dd613d40b5c675f7e8d150ce070b864d7e8df
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
import pandas as pd

df = pd.read_csv('conull.csv')


In [ ]:
import pandas as pd


# Normalize the label column
df["Label"] = df["Label"].str.strip().str.upper().str.replace(":", "", regex=False)

# Check unique values
print(df["Label"].unique())



['B-PRODUCT' 'O' 'I-PRODUCT' 'LABEL' 'B-AUDIENCE' 'B-BRAND' 'I-BRAND'
 'B-COMPONENT' 'I-COMPONENT' 'B-TASK' 'I-TASK' 'B-CONTACT_INFO' '2'
 'B-PRICE' 'B-LOC' 'I-LOC' 'B-DATE' 'I-DATE' 'B-FEATURE' 'I-FEATURE'
 'I-PRICE' 'B-ATTRIBUTE' 'I-ATTRIBUTE']


In [ ]:
label_list = [
    'O', 'B-PRODUCT', 'I-PRODUCT', 'B-AUDIENCE', 'B-BRAND', 'I-BRAND',
    'B-COMPONENT', 'I-COMPONENT', 'B-TASK', 'I-TASK', 'B-CONTACT_INFO',
    'B-PRICE', 'I-PRICE', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE',
    'B-FEATURE', 'I-FEATURE', 'B-ATTRIBUTE', 'I-ATTRIBUTE'
]


In [ ]:
# Convert labels to uppercase and strip spaces
df["Label"] = df["Label"].str.strip().str.upper().str.replace(":", "", regex=False)

# Drop invalid rows
df = df[df["Label"].isin(label_list)].reset_index(drop=True)

In [ ]:
dataset_dict = {
    "tokens": [df["A"].tolist()],
    "ner_tags": [[label_list.index(label) for label in df["Label"].tolist()]]
}



In [ ]:
df["Label"] = df["Label"].str.upper()


In [ ]:
# Convert labels to uppercase and strip spaces
df["Label"] = df["Label"].str.strip().str.upper().str.replace(":", "", regex=False)

# Drop invalid rows
df = df[df["Label"].isin(label_list)].reset_index(drop=True)

In [ ]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict({
    "train": Dataset.from_dict(dataset_dict),
    "validation": Dataset.from_dict(dataset_dict)
})


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
from transformers import AutoModelForTokenClassification
from transformers import AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=21)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "xlm-roberta-base"  # or any Amharic-supported model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)



In [ ]:
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "f1": f1_score(true_labels, true_predictions),
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "report": classification_report(true_labels, true_predictions, output_dict=False)
    }


In [ ]:
from transformers import Trainer
from transformers import TrainingArguments

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)


In [ ]:
output = trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sumeyasirmulach (sumeyasirmulach-temer-properties) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
train_loss_xlm = output.training_loss


In [ ]:
results_xlm = trainer.evaluate()

print(results_xlm)


{'eval_loss': 2.7906816005706787, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_report': '              precision    recall  f1-score   support\n\n    AUDIENCE       0.00      0.00      0.00         8\n       BRAND       0.00      0.00      0.00         1\n   COMPONENT       0.00      0.00      0.00         4\nCONTACT_INFO       0.00      0.00      0.00         9\n     PRODUCT       0.00      0.00      0.00        77\n        TASK       0.00      0.00      0.00         3\n\n   micro avg       0.00      0.00      0.00       102\n   macro avg       0.00      0.00      0.00       102\nweighted avg       0.00      0.00      0.00       102\n', 'eval_runtime': 4.2924, 'eval_samples_per_second': 0.233, 'eval_steps_per_second': 0.233, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = AutoModelForTokenClassification.from_pretrained("rasyosef/bert-tiny-amharic", num_labels=21)



config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at rasyosef/bert-tiny-amharic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "rasyosef/bert-tiny-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/274k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/725k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

In [ ]:
output = trainer.train()


Step,Training Loss


In [ ]:
train_loss_bert_tiny = output.training_loss


In [ ]:
results_bert_tiny = trainer.evaluate()
print(results_bert_tiny)

{'eval_loss': 2.8496713638305664, 'eval_f1': 0.037333333333333336, 'eval_precision': 0.026415094339622643, 'eval_recall': 0.06363636363636363, 'eval_report': '              precision    recall  f1-score   support\n\n   ATTRIBUTE       0.00      0.00      0.00         0\n    AUDIENCE       0.00      0.00      0.00         0\n       BRAND       0.00      0.00      0.00         0\n   COMPONENT       0.00      0.00      0.00         0\nCONTACT_INFO       0.00      0.00      0.00         0\n        DATE       0.00      0.00      0.00         0\n     FEATURE       0.00      0.00      0.00         0\n         LOC       0.00      0.00      0.00         0\n     PRODUCT       0.10      0.06      0.08       110\n        TASK       0.00      0.00      0.00         0\n\n   micro avg       0.03      0.06      0.04       110\n   macro avg       0.01      0.01      0.01       110\nweighted avg       0.10      0.06      0.08       110\n', 'eval_runtime': 0.0706, 'eval_samples_per_second': 14.161, 'eval

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0",
    num_labels=21,
    ignore_mismatched_sizes=True  # 👈 key fix
)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([21]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([21, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/404 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=2.8555733362833657, metrics={'train_runtime': 0.5251, 'train_samples_per_second': 5.714, 'train_steps_per_second': 5.714, 'total_flos': 3681874944.0, 'train_loss': 2.8555733362833657, 'epoch': 3.0})

In [ ]:
output = trainer.train()

train_loss_afroxlmr = output.training_loss


Step,Training Loss


In [ ]:
results_afroxlmr = trainer.evaluate()
print(results_afroxlmr)

{'eval_loss': 2.7928833961486816, 'eval_f1': 0.04833836858006042, 'eval_precision': 0.03619909502262444, 'eval_recall': 0.07272727272727272, 'eval_report': '              precision    recall  f1-score   support\n\n   ATTRIBUTE       0.00      0.00      0.00         0\n    AUDIENCE       0.00      0.00      0.00         0\n       BRAND       0.00      0.00      0.00         0\n   COMPONENT       0.00      0.00      0.00         0\nCONTACT_INFO       0.00      0.00      0.00         0\n     FEATURE       0.00      0.00      0.00         0\n         LOC       0.00      0.00      0.00         0\n     PRODUCT       0.09      0.07      0.08       110\n        TASK       0.00      0.00      0.00         0\n\n   micro avg       0.04      0.07      0.05       110\n   macro avg       0.01      0.01      0.01       110\nweighted avg       0.09      0.07      0.08       110\n', 'eval_runtime': 0.0392, 'eval_samples_per_second': 25.525, 'eval_steps_per_second': 25.525, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd

comparison_data = {
    "Model": ["XLM-Roberta", "BERT-Tiny-Amharic", "AfroXLMR"],
    "Train Loss": [train_loss_xlm, train_loss_bert_tiny, train_loss_afroxlmr],
    "Eval F1": [
        results_xlm.get("eval_f1", None),
        results_bert_tiny.get("eval_f1", None),
        results_afroxlmr.get("eval_f1", None)
    ],
    "Precision": [
        results_xlm.get("eval_precision", None),
        results_bert_tiny.get("eval_precision", None),
        results_afroxlmr.get("eval_precision", None)
    ],
    "Recall": [
        results_xlm.get("eval_recall", None),
        results_bert_tiny.get("eval_recall", None),
        results_afroxlmr.get("eval_recall", None)
    ]
}

df = pd.DataFrame(comparison_data)
print(df)


               Model  Train Loss   Eval F1  Precision    Recall
0        XLM-Roberta    2.913742  0.000000   0.000000  0.000000
1  BERT-Tiny-Amharic    2.875096  0.037333   0.026415  0.063636
2           AfroXLMR    2.828847  0.048338   0.036199  0.072727


In [ ]:
from IPython.display import display
display(df.style.set_caption("NER Model Comparison").format(precision=3))


,Model,Train Loss,Eval F1,Precision,Recall
0,XLM-Roberta,2.914,0.000,0.000,0.000
1,BERT-Tiny-Amharic,2.875,0.037,0.026,0.064
2,AfroXLMR,2.829,0.048,0.036,0.073


In [ ]:
text = "አዲስ አበባ ዉስጥ ከ150ብር እስከ 200ብር ብቻ በማስከፈል ያሉበት ድረስ በፈጣን ሞተረኞቻችን እንልክልዏታለን። "


In [ ]:
!pip install lime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=b27f386edaa0ce39e60e1fdf4ee939ad8ce0fbd66dcf37383b0d255bcf950f13
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime


In [ ]:
from transformers import pipeline
from lime.lime_text import LimeTextExplainer

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
explainer = LimeTextExplainer(class_names=label_list)  # your label list from earlier


Device set to use cpu


In [ ]:
def fast_predictor(texts):
    outputs = []
    for t in texts:
        pred = ner_pipeline(t)
        # Simplify: just score if any PRODUCT/PRICE was detected
        score = [0] * len(label_list)
        for p in pred:
            if p['entity_group'] in label_list:
                score[label_list.index(p['entity_group'])] += p['score']
        outputs.append(score)
    return outputs



In [ ]:
import shap

explainer = shap.Explainer(model, tokenized_train)
shap_values = explainer(tokenized_test)
shap.summary_plot(shap_values, tokenized_test['tokens'])

NameError: name 'tokenized_train' is not defined

In [ ]:
exp = explainer.explain_instance(text, fast_predictor, num_features=6)
exp.show_in_notebook()


TypeError: list indices must be integers or slices, not tuple